## 머신러닝

### 텍스트 마이닝

#### 개념

##### 텍스트 마이닝
- 비정형 텍스트 데이터에서 패턴을 찾음. 의미가 있는 정보들을 추출하고 분석하는 기법
- 데이터 마이닝, 자연어 처리, 정보 검색 등의 분야가 결합된 분석기법 미용
- 프로세스 : 텍스트 전처리 > 특성 벡터화 > 머신러닝 모델 구축, 학습, 평가
- 텍스트 전처리 : 토큰화, 불용어 제거, 표제어 추출, 형태소 분석...

##### 특성 벡터화/추출
- 컴퓨터는 글을 읽을 수 없음 -> 단어기반으로 특성 추출, 숫자형 벡터값으로 표현
- BowW, Word2ve 라이브러리(모듈)존재

##### LDA
- 문서에 잠재되어있는 토픽을 추론하는 확률 모델 알고리즘
- pyLSAvis 시각화 라이브러리 사용

#### 데이터 수집
- https://github.com/e9t/nsmc
- ratings.txt, ratings_test.txt, ratings_train.txt 다운로드

- id(리뷰번호), document(리뷰), label(감성분류 클래스 0은 부정, 1은 긍정 감성)

In [1]:
## 필요 라이브러리 사용등록
import pandas as pd
import re

In [2]:
# 모든 모듈/라이브러리는 __version__
pd.__version__

'2.2.1'

In [3]:
## pandas 버전 상세정보
pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : bdc79c146c2e32f2cab629be240f01658cfb6cc2
python                : 3.12.2.final.0
python-bits           : 64
OS                    : Windows
OS-release            : 11
Version               : 10.0.22000
machine               : AMD64
processor             : Intel64 Family 6 Model 94 Stepping 3, GenuineIntel
byteorder             : little
LC_ALL                : None
LANG                  : None
LOCALE                : Korean_Korea.949

pandas                : 2.2.1
numpy                 : 1.26.4
pytz                  : 2024.1
dateutil              : 2.8.2
setuptools            : 69.1.1
pip                   : 24.0
Cython                : None
pytest                : None
hypothesis            : None
sphinx                : None
blosc                 : None
feather               : None
xlsxwriter            : None
lxml.etree            : 5.1.0
html5lib              : None
pymysql               : None
psycopg2              : Non

In [4]:
## 훈련용 데이터 가져오기
dfNsmcTrain = pd.read_csv('./data/ratings_train.txt', engine='python', sep='\t', encoding='utf-8')

In [5]:
dfNsmcTrain

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [6]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
dfNsmcTrain = dfNsmcTrain[ dfNsmcTrain['document'].notnull()]

In [8]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149995 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149995 non-null  int64 
 1   document  149995 non-null  object
 2   label     149995 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [9]:
dfNsmcTrain['label'].value_counts()

label
0    75170
1    74825
Name: count, dtype: int64

In [10]:
## Ragular Expression(정규식)으로 한글 이외의 문제는 다 제거
dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ|가-힣]+', ' ', x))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11696\2606043373.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNsmcTrain['document'] = dfNsmcTrain['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ|가-힣]+', ' ', x))


In [11]:
dfNsmcTrain.head()


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [12]:
dfNsmcTrain.tail()

,id,document,label
149995,6222902,인간이 문제지 소는 뭔죄인가,0
149996,8549745,평점이 너무 낮아서,1
149997,9311800,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
149998,2376369,청춘 영화의 최고봉 방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [13]:
## 평가용 데이터 준비
dfNsmcTest = pd.read_csv('./data/ratings_test.txt', engine='python', sep='\t', encoding='utf-8')

In [14]:
dfNsmcTest.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [17]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [18]:
dfNsmcTest[dfNsmcTest['document'].isnull()]

,id,document,label
5746,402110,NaN,1
7899,5026896,NaN,0
27097,511097,NaN,1


In [19]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'].notnull()]

In [20]:
dfNsmcTest['label'].value_counts()

label
1    25171
0    24826
Name: count, dtype: int64

In [21]:
dfNsmcTest['document'] = dfNsmcTest['document'].apply(lambda x: re.sub(r'[^ ㄱ-ㅣ|가-힣]+', ' ', x))

In [22]:
dfNsmcTest

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,,0
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
49997,9072549,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,0


In [23]:
## 한글 이외의 텍스트 삭제 후 document가 비어있는 ' ' 데이터를 제거
dfNsmcTrain = dfNsmcTrain[dfNsmcTrain['document'] != ' ']

In [24]:
dfNsmcTrain.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149186 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        149186 non-null  int64 
 1   document  149186 non-null  object
 2   label     149186 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [25]:
dfNsmcTest = dfNsmcTest[dfNsmcTest['document'] != ' ']

In [26]:
dfNsmcTest.info()

<class 'pandas.core.frame.DataFrame'>
Index: 49726 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        49726 non-null  int64 
 1   document  49726 non-null  object
 2   label     49726 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


##### 분석모델 구축

- 이전, 특징 벡터화 작업, 단어로 분리(토큰화) 후 TF-IDF 방식으로 벡터화
- 한글 형태소 분석은 konlpy 패키지의 Okt 클래스 사용

##### KoNLPy 라이브러리 설치
- JDK 설치
- 시스템 등록정보 JAVA-HOME 설정
    - 프로그램 실행 sysdm.cpl
    - 시스템속성 > 고급탭 > 환경변수 버튼 클릭
    - 시스템변수 > 새로만들기 버튼 클릭
    - JAVA_HOME과 경로 추가등록
- JPype1 : 파이썬에서 JDK를 사용하게 해주는 프로그램
- koNLPy 설치

In [27]:
# JPype1 설치
!pip install JPype1

In [28]:
# koNLPy 설치
!pip install koNLPy

In [29]:
import konlpy

In [30]:
konlpy.__version__

'0.6.0'

In [31]:
## 특성 벡터화 작업
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [32]:
okt = Okt()

In [34]:
def oktTokenizer(text):
    tokens = okt.morphs(text)
    return tokens

In [35]:
tfidf = TfidfVectorizer(tokenizer=oktTokenizer, ngram_range=(1, 2), min_df=3, max_df=0.9)

In [36]:
tfidf.fit(dfNsmcTrain['document'])
nsmc_train_tfidf = tfidf.transform(dfNsmcTrain['document'])

c:\DEV\Python312\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [37]:
nsmc_train_tfidf

<149186x115705 sparse matrix of type '<class 'numpy.float64'>'
	with 2712786 stored elements in Compressed Sparse Row format>

In [38]:
with open('result.npy', 'wb') as f:
    np.save(f, nsmc_train_tfidf)

In [39]:
print(nsmc_train_tfidf)

  (0, 100754)	0.45381032241388186
  (0, 100230)	0.5246405777842206
  (0, 99782)	0.18463947050544674
  (0, 58187)	0.21270642657406902
  (0, 38522)	0.3275840599924698
  (0, 24760)	0.48400664174037533
  (0, 24733)	0.31288808195337253
  (1, 115300)	0.22359223094672936
  (1, 107553)	0.28646957238552473
  (1, 107538)	0.19941904793427181
  (1, 102737)	0.2931900792257784
  (1, 102725)	0.20771374470518777
  (1, 97714)	0.17472448538619545
  (1, 95359)	0.22409419947584006
  (1, 71271)	0.291960626293841
  (1, 71264)	0.23964814595542253
  (1, 70339)	0.34043273510350347
  (1, 69437)	0.06844745838463881
  (1, 68565)	0.34043273510350347
  (1, 68435)	0.12762958490485107
  (1, 60934)	0.347153241943757
  (1, 43815)	0.13593256179129157
  (1, 2729)	0.28964275871334305
  (2, 110512)	0.2596717913549582
  (2, 110409)	0.12692407466606534
  :	:
  (149184, 73754)	0.33487457422468325
  (149184, 70164)	0.17282615986750355
  (149184, 69437)	0.06602649986783408
  (149184, 42367)	0.3230949313689824
  (149184, 42366)	

##### 감성 분류모델 구축

In [40]:
## 로지스틱회귀 라이브러리 등록
from sklearn.linear_model import LogisticRegression

In [41]:
model = LogisticRegression(random_state=0)

In [42]:
## 감성 이진분류 모델 훈련
model.fit(nsmc_train_tfidf, dfNsmcTrain['label'])

LogisticRegression(random_state=0)

In [43]:
# 로지스틱회귀 사용될 하이퍼 파라미터 리스트
model.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [44]:
## 최적 예측모델을 찾기위한 작업
# 추가 라이브러리 사용등록
from sklearn.model_selection import GridSearchCV

In [45]:
## 하이퍼 파라미터 중 C(현재 1.0)값을 변경하면서 최적예측 모델 찾기
params = {'C': [3.0, 3.5, 4.0, 4.5, 5.0]}
testModel = GridSearchCV(model, param_grid=params, cv=3, scoring='accuracy', verbose=1)

In [46]:
testModel.fit(nsmc_train_tfidf, dfNsmcTrain['label'])

Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\DEV\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [3.0, 3.5, 4.0, 4.5, 5.0]}, scoring='accuracy',
             verbose=1)

In [48]:
# best_params_, best_score_ 출력 불가
testModel.best_estimator_

LogisticRegression(C=3.5, random_state=0)

In [49]:
# C=3.5로 해서 bestModel을 생성
bestModel = testModel.best_estimator_

##### 검증(평가)용 데이터 벡터화

In [50]:
nsmc_train_tfidf = tfidf.transform(dfNsmcTest['document'])

In [52]:
# 예측결과
y_predict = bestModel.predict(nsmc_train_tfidf)

In [53]:
# 감정분류 예측결과
len(y_predict)

49726

In [54]:
# 실제 라벨 수 동일
len(dfNsmcTest['label'])

49726

In [51]:
# 평가지표로 정확도 확인
from sklearn.metrics import accuracy_score

In [56]:
# 베스트 모델로 예측결과
accuracy_score(dfNsmcTest['label'], y_predict)

0.8598519888991675

##### 베스트모델로 감성 예측

In [57]:
sentence = input('리뷰 입력 >> ')

In [58]:
sentence

'오늘은 좋은 날이 될 것 같은 예감 100%'

In [59]:
# 입력 문장에서 한글 외의 것을 제거하고 한글만 추출
st = re.sub(r'[^가-힣|ㄱ-]+', ' ', sentence)
st

'오늘은 좋은 날이 될 것 같은 예감 '

In [60]:
## 리스트로 형변환
lstSt = [st]
lstSt

['오늘은 좋은 날이 될 것 같은 예감 ']

In [61]:
# 입력 텍스트의 벡터화
st_tfidf = tfidf.transform(lstSt)
print(st_tfidf)

  (0, 96584)	0.36940010574154
  (0, 96566)	0.16567175262274383
  (0, 81296)	0.27422419226400396
  (0, 80652)	0.07323854035317007
  (0, 76877)	0.32940700119847816
  (0, 76044)	0.10924224147831563
  (0, 71099)	0.30873610017855524
  (0, 71068)	0.21467804658712675
  (0, 70678)	0.3275390584688332
  (0, 27506)	0.2841027616467524
  (0, 27501)	0.21537462928353554
  (0, 16309)	0.30399153745287066
  (0, 16280)	0.22037319801448751
  (0, 6334)	0.26664889213564863
  (0, 6310)	0.13937194963319052
  (0, 4575)	0.1667403359677299


In [63]:
# 0은 부정적인 감정, 1은 긍정적인 감정
str_predict = bestModel.predict(st_tfidf)

In [64]:
str_predict

array([1], dtype=int64)

##### 결론
- 감정예측은 오류가 많을 수 있음
- 훈련도 많이 시켜야 하고, 머신러닝을 딥러닝과 결합해서 더 정확도를 높여야 함